In [1]:
import boto3

bucket_list_dataStationInformation = []

s3 = boto3.resource('s3')
s3_object = s3.Bucket('citibikejsonfiles')

for file in s3_object.objects.filter(Prefix = 'dataStationInformation'):
    file_name = file.key
    if file_name.find(".json") != -1:
        bucket_list_dataStationInformation.append(file.key)

len_bucket_list_dataStationInformation = print(len(bucket_list_dataStationInformation))
print(bucket_list_dataStationInformation[0:10])


998
['dataStationInformation 2023-01-14 01:40:37.364932.json', 'dataStationInformation 2023-01-14 01:50:37.351757.json', 'dataStationInformation 2023-01-14 02:00:37.354085.json', 'dataStationInformation 2023-01-14 02:10:37.297757.json', 'dataStationInformation 2023-01-14 02:20:37.491889.json', 'dataStationInformation 2023-01-14 02:30:37.376058.json', 'dataStationInformation 2023-01-14 02:40:37.058998.json', 'dataStationInformation 2023-01-14 02:50:37.666289.json', 'dataStationInformation 2023-01-14 03:00:37.288828.json', 'dataStationInformation 2023-01-14 03:10:37.558505.json']


In [2]:
# To import dataStationStatus s3 files

bucket_list_dataStationStatus = []

for file in s3_object.objects.filter(Prefix = 'dataStationStatus'):
    file_name_status = file.key
    if file_name_status.find(".json") != -1:
        bucket_list_dataStationStatus.append(file.key)

len_bucket_list_dataStationStatus = print(len(bucket_list_dataStationStatus))
print(bucket_list_dataStationStatus[0:10])

998
['dataStationStatus 2023-01-14 01:40:36.938626.json', 'dataStationStatus 2023-01-14 01:50:36.909127.json', 'dataStationStatus 2023-01-14 02:00:36.817121.json', 'dataStationStatus 2023-01-14 02:10:36.800837.json', 'dataStationStatus 2023-01-14 02:20:37.034204.json', 'dataStationStatus 2023-01-14 02:30:36.863208.json', 'dataStationStatus 2023-01-14 02:40:36.607410.json', 'dataStationStatus 2023-01-14 02:50:37.224251.json', 'dataStationStatus 2023-01-14 03:00:36.820247.json', 'dataStationStatus 2023-01-14 03:10:37.096055.json']


In [3]:
# To get the timestamp key for DataStationInformation
dataframe_key_dataStationInformation = []

for file in bucket_list_dataStationInformation:
    file_name = file[23:39]
    dataframe_key_dataStationInformation.append(file_name)

dataframe_key_dataStationInformation[:5]

['2023-01-14 01:40',
 '2023-01-14 01:50',
 '2023-01-14 02:00',
 '2023-01-14 02:10',
 '2023-01-14 02:20']

In [4]:
# To get the timestamp key for DataStationStatus
dataframe_key_dataStationStatus = []

for file in bucket_list_dataStationStatus:
    file_name = file[-31:-15] # need to update
    dataframe_key_dataStationStatus.append(file_name)

dataframe_key_dataStationStatus[:5]

['2023-01-14 01:40',
 '2023-01-14 01:50',
 '2023-01-14 02:00',
 '2023-01-14 02:10',
 '2023-01-14 02:20']

In [5]:
from io import StringIO
import io
import pandas as pd

dataframe_dataStationInformation = [] # empty list of dataframes

for file in bucket_list_dataStationInformation:
    s3_obj = s3.Object('citibikejsonfiles', file)
    data = s3_obj.get()['Body'].read()
    dataframe_dataStationInformation.append(pd.read_json(io.BytesIO(data)))

In [ ]:
dataframe_dataStationStatus = [] # empty list of dataframes

for file in bucket_list_dataStationStatus:
    s3_obj = s3.Object('citibikejsonfiles', file)
    data = s3_obj.get()['Body'].read()
    dataframe_dataStationStatus.append(pd.read_json(io.BytesIO(data)))

In [8]:
# StationInformationTimeSeries is the DataFrame

appendedDataStationInformation = []
for key in range(len(dataframe_dataStationInformation)):
    tmpDataStationInformation = pd.json_normalize(dataframe_dataStationInformation[key]['data']['stations'])
    tmpDataStationInformation['time_stamp'] = dataframe_key_dataStationInformation[key]
    appendedDataStationInformation.append(tmpDataStationInformation)
StationInformationTimeSeries = pd.concat(appendedDataStationInformation, ignore_index = True)

# 'time_stamp', 'station_id', 'name', 'short_name', 'capacity', 'has_kiosk', 'region_id', 'lat', 'lon'

In [ ]:
# StationStatusTimeSeries is the DataFrame

appendedDataStationStatus = []

for key in range(len(dataframe_dataStationStatus)):
    tmpDataStationStatus = pd.json_normalize(dataframe_dataStationStatus[key]['data']['stations'])
    tmpDataStationStatus['time_stamp'] = dataframe_key_dataStationStatus[key]
    appendedDataStationStatus.append(tmpDataStationStatus)
StationStatusTimeSeries = pd.concat(appendedDataStationStatus, ignore_index = True)

# 'time_stamp', 'station_id', 'station_status', 'num_bikes_available', 'num_bikes_disabled',
# 'num_docks_available', 'num_docks_disabled', 'num_ebikes_available', 'is_renting',
# 'is_returning', 'is_installed', 'last_reported'

In [ ]:
# Dropping a weird station identified as 4987 that returned NaN results and may be an offline station
# Last code line is the finalized columns for the DataFrame

StationInformationTimeSeries.drop(StationInformationTimeSeries[(StationInformationTimeSeries.lat == 0.0) & (StationInformationTimeSeries.lon == 0.0)].index, inplace = True)

StationInformationTimeSeries = StationInformationTimeSeries[['time_stamp', 'station_id', 'name', 'short_name', 'capacity', 'has_kiosk', 'region_id', 'lat', 'lon']]

In [ ]:
# Dropping a weird station identified as 4987 that returned NaN results and may be an offline station
# Last code line is the finalized columns for the DataFrame

StationStatusTimeSeries.drop(StationStatusTimeSeries[StationStatusTimeSeries.station_id == '4987'].index, inplace = True)

StationStatusTimeSeries = StationStatusTimeSeries[['time_stamp', 'station_id', 'station_status', 'num_bikes_available', 'num_bikes_disabled', 'num_docks_available', 'num_docks_disabled', 'num_ebikes_available', 'is_renting', 'is_returning', 'is_installed', 'last_reported']]